In [73]:
import cv2
import numpy as np
import time
import pickle

drawing = False # true if mouse is pressed
ix,iy = -1,-1
p = 2
ct = 1
parking_coordinates = []
pts = None
vidID = 'Day2.mp4'

# mouse callback function
def draw_points(event,x,y,flags,param):
    global ix,iy, drawing
    global pts

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
        parking_coordinates.append([x,y])
        if(len(parking_coordinates) ==4):
            #global pts
            pts = np.array(parking_coordinates, dtype = np.int32)
            pts = pts.reshape((-1,1,2))
            mapping = {vidID:pts}
            print(mapping)
            mapping_file = open('mapping_file.pkl', 'wb')
            pickle.dump(mapping, mapping_file)

In [74]:
vidcap = cv2.VideoCapture('Day2.mp4')
success,frame = vidcap.read()
cv2.namedWindow('image')

while(success):
    success, frame = vidcap.read()
    if(frame is not None):
        height, width, layers = frame.shape
        new_h = int(height/2)
        new_w = int(width/2)
        resize = cv2.resize(frame, (new_w, new_h)) 
        cv2.setMouseCallback('image',draw_points)
        cv2.polylines(frame, [pts], True,(0,0,255), thickness= 5)
        cv2.imshow('image',frame)
        
        if(pts is not None):
            if(len(pts) == 4):
                time.sleep(2)
                break
        
        
    else:
        vidcap = cv2.VideoCapture(vidID)
        success,frame = vidcap.read()
        continue
        
#     i=0
#     #draw the polygon
#     while i<parking_coordinates.__len__():
#         frame[int(0.95*parking_coordinates[i][0]):int(1.05*parking_coordinates[i][0]),int(0.95*parking_coordinates[i][1]):int(1.05*parking_coordinates[i][1])]=(0,0,255)
#         i+=1
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
vidcap.release()
cv2.destroyAllWindows()

{'Day2.mp4': array([[[1285,  680]],

       [[ 709,  567]],

       [[1409,  196]],

       [[1704,  258]]], dtype=int32)}


# Using PIL

In [75]:
from PIL import Image
from PIL import ImageDraw 
from moviepy.editor import VideoFileClip

def draw_parking_area(image_np):
    
    pkl_file = open(parking_map_file, 'rb')
    mapping_file = pickle.load(pkl_file)
    pkl_file.close()
    global mapping
    mapping = list(mapping_file.values())[0]
    mapping = mapping.tolist()
    mapping = list(map(lambda x: tuple(x[0]), mapping))
    
    image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
    draw = ImageDraw.Draw(image_pil)
    mapping.append(mapping[0]) 
    #print(mapping)
    
    draw.line(mapping, fill="red", width=9)
    for point in mapping:
        draw.ellipse((point[0] - 4, point[1] - 4, point[0]  + 4, point[1] + 4), fill="red")
    
        
    #draw.polygon(mapping, fill=None, outline=128, thickness = 100)
    np.copyto(image_np, np.array(image_pil))
    
    return image_np
    

In [76]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image with lines are drawn on lanes
    image_process = draw_parking_area(image)
    return image_process

In [77]:
white_output = 'testVideo_out2.mp4'
vidFile = "Day2.mp4"
parking_map_file = 'mapping_file.pkl'
clip1 = VideoFileClip(vidFile).subclip(0, 5)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False, codec='mpeg4')
print(mapping)

[MoviePy] >>>> Building video testVideo_out2.mp4
[MoviePy] Writing video testVideo_out2.mp4


 99%|█████████▉| 125/126 [00:05<00:00, 24.93it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: testVideo_out2.mp4 

CPU times: user 5.98 s, sys: 1.65 s, total: 7.63 s
Wall time: 5.09 s
[(1285, 680), (709, 567), (1409, 196), (1704, 258), (1285, 680)]
